In [47]:
from datasets import load_dataset

dataset = load_dataset("squad")

In [1]:
import spacy
nlp = spacy.load('en_core_web_trf')

/Users/seongilpark/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [129]:
import json

with open("nq-test.json", "r") as f:
    nq = json.load(f)

In [130]:
nq_questions = [e['question'] for e in nq]

In [131]:
docs = list(nlp.pipe(nq_questions,batch_size=128))

In [132]:
def masking_entity(doc) -> str:
    if not len(doc.ents):
        return doc.text
    
    text_list =[]
    for d in doc:
        if d.pos_ == "PUNCT":
            text_list.append("@"+d.text)
        elif d.pos_ == "AUX" and d.text == "'s":
            text_list.append("@"+d.text)
        else:
            text_list.append(d.text)

    for ent in doc.ents:
        text_list[ent.start:ent.end] = ["[B]"]* (ent.end - ent.start)
        text_list[ent.start] = "[BLANK]"
    return " ".join(text_list).replace(" [B]", "").replace(" @", "")

In [133]:
result = []
for doc in docs:
    try:
        result.append(masking_entity(doc))
    except:
        print(doc)

In [134]:
print(len(nq_questions))
print(len(result))

3610
3610


In [136]:
from tqdm import tqdm

emds = []
for i in tqdm(range(0, len(result), 32)):
    batch = result[i:i+32]
    embeddings = model.encode(batch)
    for idx, emb in enumerate(embeddings):
        emds.append(emb)

100%|██████████| 113/113 [00:04<00:00, 26.24it/s]


In [140]:
for q, r, e in zip(nq, result, emds):
    q['masked_question'] = r
    q['masked_question_embedding'] = e.tolist()
with open("nq-test-with-embds.json", "w") as f:
    json.dump(nq, f)

In [142]:
nq[0]['masked_question_embedding']

[-0.050751253962516785,
 0.0415990948677063,
 0.004519982263445854,
 0.11377488076686859,
 -0.025766627863049507,
 -0.00020134275837335736,
 0.010921486653387547,
 0.005758862942457199,
 0.053729988634586334,
 -0.013639156706631184,
 0.09257633239030838,
 -0.04823893681168556,
 -0.05879274010658264,
 -0.02404460869729519,
 -0.0478200763463974,
 0.05612983927130699,
 -0.03818182274699211,
 0.0011802057269960642,
 -0.08060953766107559,
 -0.04245540499687195,
 -0.008635022677481174,
 0.04528072476387024,
 0.024225464090704918,
 -0.02777707204222679,
 0.048539645969867706,
 0.04327670857310295,
 -0.0222444087266922,
 -0.045138731598854065,
 0.06330950558185577,
 0.01619591750204563,
 0.02565889060497284,
 0.026625975966453552,
 -0.004932265728712082,
 -0.03327135741710663,
 0.051182471215724945,
 -0.007155284751206636,
 0.07120928168296814,
 0.023139819502830505,
 -0.019660739228129387,
 -0.0036644835490733385,
 -0.006209670100361109,
 -0.001931150909513235,
 0.03564820811152458,
 0.096628

In [118]:
buffer = dict()
result_idxs = []
for idx, sent in enumerate(result):
    if not buffer.get(sent):
        buffer[sent] = 1
        result_idxs.append(idx)

In [119]:
temp = []
for idx, t in enumerate(train):
    if idx in result_idxs:
        t['masked_question'] = result[idx]
        temp.append(t)
    else:
        continue

In [120]:
import json
with open("masked_train.json", "w") as f:
    json.dump(temp, f)

In [109]:
not buffer.get('Which [BLANK] team represented the [BLANK] at [BLANK]?')

False

In [42]:
import json

In [121]:
with open("masked_train.json", "r") as f:
    train = json.load(f)
with open("masked_test.json", "r") as f:
    test = json.load(f)
total = train + test
len(total)

93544

In [122]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [125]:
from tqdm import tqdm

for i in tqdm(range(0, len(total), 32)):
    batch = total[i:i+32]
    sents = [b['masked_question'] for b in batch]
    embeddings = model.encode(sents)
    for idx, emb in enumerate(embeddings):
        total[i+idx]['embedding'] = emb

100%|██████████| 2924/2924 [01:55<00:00, 25.23it/s]


In [126]:
import joblib
joblib.dump(total, "total_squad_with_embedding")

['total_squad_with_embedding']

In [127]:
total[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]},
 'masked_question': 'To whom did [BLANK] allegedly

In [100]:
from collections import Counter

doc_count = Counter(result)

In [102]:
doc_count['Where is [BLANK] located?']

11

In [1]:
from utils import evaluate_gpt
import json
with open("nq-test.json", "r") as f:
    data = json.load(f)

In [2]:
evaluate_gpt(data, num_docs=0, output_dir="result",max_tokens_to_generate=60, is_test=True, max_test=70)

EM:  0.0%:   0%|          | 0/70 [00:00<?, ?it/s]

Sample :
 Answer these questions:
Q: when is the publishers clearing house sweepstakes drawing?
A: just after the Super Bowl
Q: who got the first nobel prize in physics?
A:


EM: 15.7%: 100%|██████████| 70/70 [00:38<00:00,  1.83it/s]

EM: 15.7%
% of prompts with answer: 2.9%


In [4]:
import json
import gzip

# JSON 파일 로드
with open('masked_test.json', 'r', encoding='utf-8') as f:
    data1 = json.load(f)
with open('masked_train.json', 'r', encoding='utf-8') as f:
    data2 = json.load(f)
data = data1 + data2
del data1
del data2
# JSONL 형식으로 GZipped 파일로 저장
with gzip.open('output.jsonl.gz', 'wt', encoding='utf-8') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')


In [10]:
with gzip.open("/Users/seongilpark/rag/output.jsonl.gz", "rb") as file_handle:
    header = json.loads(file_handle.readline())            
    for entry in file_handle:
        entry = json.loads(entry)
        print(entry)
        break

{'id': '56be4db0acb8001400a502ee', 'title': 'Super_Bowl_50', 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.', 'question': 'Where did Super Bowl 50 take place?', 'answers': {'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Fran

In [11]:
import spacy

In [12]:
nlp = spacy.load("en_core_web_trf")

/Users/seongilpark/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
nlp("")